Here we will utilize our pre-processed and cleaned data set in order to answer our research question: 
### Is lending racially discriminatory in the US?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import sys

In [ ]:
# Read in csv from pre-processing. We must re-cast types because they were converted to strings upon exporting. 
df = pd.read_csv("encoded_loan_data.csv", sep=',', engine='python', error_bad_lines=False, dtype='unicode')

In [ ]:
sorted(df.applicant_race_name_1.unique())

In [ ]:
# Define immutable, related set of constant values.
import enum
from functools import total_ordering
@total_ordering
class Race(enum.Enum):
    Native = 'American Indian or Alaska Native'
    Asian = 'Asian'
    Black = 'Black or African American'
    PacificIslander = 'Native Hawaiian or Other Pacific Islander'
    White = 'White'
    nan = 'nan'
    def __lt__(self, other):
        if self.__class__ is other.__class__:
            return self.value < other.value
        return NotImplemented

Before visualizing this data we must ensure all processed data is numeric. These column types match those discussed in pre-processing.

In [ ]:
colToType = {
    "tract_to_msamd_income" : float, 
    "rate_spread" : float,
    "population" : int,
    "minority_population" : bool,
    "number_of_owner_occupied_units" : int, 
    "number_of_1_to_4_family_units" : int, 
    "loan_amount_000s" : float, 
    "hud_median_family_income" : float,
    "applicant_income_000s" : float,
    "sequence_number" : int, 
    "census_tract_number" : float, 
    "as_of_year" : int,
    "application_date_indicator" : int, 
    "applicant_race_name_1": Race,
    "applicant_race_name_2": Race,
    "applicant_race_name_3": Race,
    "applicant_race_name_4": Race,
    "applicant_race_name_5": Race,
}

In [ ]:
# Convert aforementioned column types
df_test = df
# Use Pandas drop_duplicates() as evidence that dataset is deduplicated
print("Deduplicated Valid Rows: %d\tFully Deduplicated: %r" 
      % (len(df_test), len(df_test) == len(df_test.drop_duplicates())))
print("Columns: %d" % len(df_test.columns.values))

# Convert types of columns
for colName, colType in colToType.items():
    if colType == int:
        df_test[colName] = df_test[colName].apply(lambda x: x if x != 'nan' else 0).astype(int)
    elif colType == float:
        df_test[colName] = df_test[colName].apply(lambda x: x if x != 'nan' else float('nan')).astype(float)
    elif colType == Race:
        df_test[colName] = df_test[colName].apply(lambda x: Race(x))

In [ ]:
numeric_cols = df_test.columns

In [ ]:
# Check column types were correctly converted
assert [col in ['int64','float'] for col in [df[y].dtype for y in numeric_cols]]

In [ ]:
df.applicant_race_name_1.value_counts()

Because the samples for Native Americans and Pacific Islanders are so small, we will exclude them for much of the analysis that is testing the effects of race on mortgage approval status.

In [ ]:
#Name each income bucket in order to plot data 
df["income_bracket"] = df["income_bracket"].replace('(0, 18]', 1)
df["income_bracket"] = df["income_bracket"].replace('(18, 75]', 2)
df["income_bracket"] = df["income_bracket"].replace('(75, 153]', 3)
df["income_bracket"] = df["income_bracket"].replace('(153, 233]', 4)
df["income_bracket"] = df["income_bracket"].replace('(233, 416]', 5)
df["income_bracket"] = df["income_bracket"].replace('(416, 470]', 6)
df["income_bracket"] = df["income_bracket"].replace('(470, 9999]', 7)
# Applicants with no income reported will be meaningless in our analysis and will be indicated in the -1 column.
df["income_bracket"] = df["income_bracket"].replace('nan', -1)

 ## DO WE NEED THIS 
 <span style="color:red">below</span>

In [ ]:
df["income_bracket"] = df["income_bracket"].astype('float')
df["action_taken_name"] = df["action_taken_name"].astype('float')

In [ ]:
# Separate data frames to run analyses on gender disaggregated data
df_gender_male = df.copy()
df_gender_female = df.copy()

In [ ]:
# applicant income histogram
df.income_bracket.hist(bins = range(-1,8))
plt.title('Applicant Income')
plt.xlabel('All Applicant Incomes')
plt.ylabel('Number of applicants')
plt.show()

In [ ]:
# Drop all female data
df_gender_male = df_gender_male.drop(df_gender_male[df_gender_male.applicant_sex_name == "Female"].index)

In [ ]:
# Drop all male data 
df_gender_female = df_gender_female.drop(df_gender_female[df_gender_female.applicant_sex_name == "Male"].index)

In [ ]:
# Applicant income for male histogram
df_gender_male.income_bracket.hist(bins = range(-1,8))
plt.title('Male Applicant Income')
plt.xlabel('All male applicant incomes')
plt.ylabel('Number of applicants')
plt.show()

In [ ]:
# Applicant income for women histogram
df_gender_female.income_bracket.hist(bins = range(-1,8))
plt.title('Female Applicant Income')
plt.xlabel('All Female applicant incomes')
plt.ylabel('Number of applicants')
plt.show()

In [ ]:
# Graph loan frequency of originated loan/purchased by institution for each income bracket
# disaggregated by race  
fig = plt.figure()
ax = df[(df.applicant_race_name_1 != Race.nan)]\
.groupby(["applicant_race_name_1", "income_bracket"])\
.action_taken_name.mean()\
.reset_index().set_index(["income_bracket", "applicant_race_name_1"])\
.unstack("applicant_race_name_1")\
.rename_axis([None, "applicant_race_name_1"], axis = 1).plot.bar(color = 'ygkrb')

plt.title('Loan status by income, conditional on race')
plt.xlabel('Income Bracket')
plt.ylabel('Loan Origination Status')
ax.legend(['Native American', 'Asian', 'Black or African American'
           ,'Pacific Islander','White'], bbox_to_anchor=(1.1, 1.0))
plt.show()

In [ ]:
# Graph loan frequency of originated loan/purchased by institution for each income bracket
# disaggregated by race only for Asian, Black, and White
fig = plt.figure()
ax = df[~(df.applicant_race_name_1.isin([Race.nan, Race.Native, Race.PacificIslander]))]\
.groupby(["applicant_race_name_1", "income_bracket"])\
.action_taken_name.mean()\
.reset_index().set_index(["income_bracket", "applicant_race_name_1"])\
.unstack("applicant_race_name_1")\
.rename_axis([None, "applicant_race_name_1"], axis = 1).plot.bar(color = 'gkb')

plt.title('Loan status by income, conditional on race')
plt.xlabel('Income Bracket')
plt.ylabel('Loan Origination Status')
ax.legend(['Asian', 'Black or African American', 'White'], bbox_to_anchor=(1.1, 1.0))
plt.show()

In [ ]:
# Calculate distance between asian and white, black and white, frequencies of originated loans/purchased by institution 
whiteMeans = df[(df.applicant_race_name_1 == Race.White) & (df.income_bracket > 0)].groupby(["applicant_race_name_1", "income_bracket"])\
.action_taken_name.mean().reset_index().set_index("income_bracket").drop("applicant_race_name_1", axis = 1)

asianMeans = df[(df.applicant_race_name_1 == Race.Asian) & (df.income_bracket > 0)].groupby(["applicant_race_name_1", "income_bracket"])\
.action_taken_name.mean().reset_index().set_index("income_bracket").drop("applicant_race_name_1", axis = 1)

blackMeans = df[(df.applicant_race_name_1 == Race.Black) & (df.income_bracket > 0)].groupby(["applicant_race_name_1", "income_bracket"])\
.action_taken_name.mean().reset_index().set_index("income_bracket").drop("applicant_race_name_1", axis = 1)

asianDiffRelWhites = (asianMeans - whiteMeans).mean()
blackDiffRelWhites = (blackMeans - whiteMeans).mean()

print("After controlling for income, asians were %0.2f%% less likely to get a loan approved relative to whites." % 
      (asianDiffRelWhites * 100))
print("After controlling for income, blacks were %0.2f%% less likely to get a loan approved relative to whites." %
      (blackDiffRelWhites * 100))

In [ ]:
# Same graph but exclusively for primary male applicants
fig = plt.figure()
ax = df[df.applicant_race_name_1.isin([Race.Asian, Race.Black, Race.White])]\
.groupby(["applicant_race_name_1", "income_bracket"]).action_taken_name\
.mean()\
.reset_index().set_index(["income_bracket", "applicant_race_name_1"])\
.unstack("applicant_race_name_1")\
.plot.bar(color = 'gkb')

ax.legend(["Asian", "Black or African American", "White"], bbox_to_anchor=(1.1, 1.0))
plt.title('Loan status by income, conditional on race for Male Applicants')
plt.xlabel('Income Bracket')
plt.ylabel('Loan status')
plt.show()

In [ ]:
# Same graph but exclusively for primary female applicants
fig = plt.figure()
ax = df_gender_female[df_gender_female.applicant_race_name_1.isin([Race.Asian, Race.Black, Race.White])]\
.groupby(["applicant_race_name_1", "income_bracket"]).action_taken_name\
.mean()\
.reset_index().set_index(["income_bracket", "applicant_race_name_1"])\
.unstack("applicant_race_name_1")\
.plot.bar(color = 'gkb')

plt.title('Loan status by income, conditional on race for Female Applicants')
plt.xlabel('Income Bracket')
plt.ylabel('Loan status')
ax.legend(['Asian', 'Black or African American','White'], bbox_to_anchor=(1.1, 1.0))
plt.show()

Looking at the mean number of application results for each race, we see for any income bracket, aside from one of the higher income brackets, there is a consistent discrepancy between loan status and race. Black or African American applicants were denied loan applications more frequently than white applicants. The amount by which the number of applicants with originated loans differ, is inconsistent between different races and across different income brackets.

The number of Native Hawaiian or Other Pacific Islander and American Indian applicants is low. As a result the confidence intervals for the following plot will be too large in order to gauage meaningful analyses with regards to race and application status?

In [ ]:
#Loan status proportions and counts by applicant race and income bracket.
p_afram = df[df.applicant_race_name_1== Race.Black].groupby('income_bracket').action_taken_name.mean()
n_afram = df[df.applicant_race_name_1== Race.Black].groupby('income_bracket').action_taken_name.count()

p_white = df[df.applicant_race_name_1== Race.White].groupby('income_bracket').action_taken_name.mean()
n_white = df[df.applicant_race_name_1== Race.White].groupby('income_bracket').action_taken_name.count()

p_asian = df[df.applicant_race_name_1== Race.Asian].groupby('income_bracket').action_taken_name.mean()
n_asian = df[df.applicant_race_name_1== Race.Asian].groupby('income_bracket').action_taken_name.count()

In [ ]:
n_afram

In [ ]:
#Plot the proportions with 1-SD error bars
fig = plt.figure()
ax = plt.subplot(111)
x = np.array([-1] + list(range(1,8)))

sd_asian = 1.96*np.sqrt(p_asian*(1-p_asian)/n_asian)
plt.vlines(x+.2, p_asian+sd_asian, p_asian-sd_asian, color='green')
a1 = plt.scatter(x + 0.2, p_asian, color='green', label="Asian")

sd_afram = 1.96*np.sqrt(p_afram*(1-p_afram)/n_afram)
plt.vlines(x, p_afram+sd_afram, p_afram-sd_afram, color='black')
b1 = plt.scatter(x, p_afram, color='black', label="Black")

sd_white = 1.96*np.sqrt(p_white*(1-p_white)/n_white)
plt.vlines(x+.1, p_white+sd_white, p_white-sd_white, color='blue')
c1 = plt.scatter(x + 0.1, p_white, color='blue', label="White")

# a2 = plt.plot(x[1:] + 0.2, p_asian[1:], color = 'green')
# b2 = plt.plot(x[1:], p_afram[1:], color='black')
# c2 = plt.plot(x[1:] + 0.1, p_white[1:], color='blue')

plt.ylabel('Loan Origination Status')
plt.xlabel('Applicant Income Bracket')
plt.xticks(range(-1,8))
ax.legend(["Asian", "Black", "White"], bbox_to_anchor=(1.05, 1.0))
plt.show()

In [ ]:
# Abe Gong's hypothetical no bias plot
fig = plt.figure()
ax = plt.subplot(111)
X = np.arange(-1,7)
plt.plot(X, 100/(1+np.exp(.5*(5-X+np.random.uniform(size=8)))), color='blue')
plt.plot(X, 100/(1+np.exp(.5*(5-X+np.random.uniform(size=8)))), color='black')
plt.plot(X, 100/(1+np.exp(.5*(5-X+np.random.uniform(size=8)))), color='green')
plt.title("Hypothetical conditional effect plot with no racial bias")
plt.ylabel('Loan Origination Status')
plt.xlabel('Applicant Income Bracket')
plt.xlim(0,max(X) + 1)
plt.ylim(0,100)
# plt.legend(["White", "Black or AFAM", "Asian"], loc='upper right')
ax.legend(["White", "Black or AFAM", "Asian"], bbox_to_anchor=(1.4, 1.0))
plt.show()

In [ ]:
# Abe Gong's hypothetical biased plot
fig = plt.figure()
ax = plt.subplot(111)
X = np.arange(-1,7)
plt.plot(X, 100/(1+np.exp(.5*(3-X+np.random.uniform(size=8)))), color='blue')
plt.plot(X, 100/(1+np.exp(.5*(5-X+np.random.uniform(size=8)))), color='black')
plt.plot(X, 100/(1+np.exp(.5*(5-X+np.random.uniform(size=8)))), color='green')
plt.title("Hypothetical conditional effect plot, biased against blacks or afam")
plt.ylabel('Loan Origination Status')
plt.xlabel('Applicant Income Bracket')
plt.xlim(0,12)
plt.ylim(0,100)
ax.legend(["White", "Black or AFAM", "Asian"], bbox_to_anchor=(1.4, 1.0))
plt.show()

# I don't even think we these two graphs below

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
X = np.arange(-1,7)
#what is X2 numbers probably aren't right for this
X2 = np.array([1,1,1,1,0,0,0,0,])
plt.plot(X, 100/(1+np.exp(.5*(3-X+np.random.uniform(size=8)))), color='blue')
plt.plot(X, 100/(1+np.exp(.5*(3-X+3*X2+np.random.uniform(size=8)))), color='black')
plt.plot(X, 100/(1+np.exp(.5*(3-X+3*X2+np.random.uniform(size=8)))), color='green')
plt.title("Hypothetical conditional effect plot, biased against low-risk black applicants")
plt.ylabel('Loan Origination Status')
plt.xlabel('Applicant Income Bracket')
plt.xlim(0,max(X) + 1)
plt.ylim(0,100)
ax.legend(["White", "Black or AFAM", "Asian"], bbox_to_anchor=(1.4, 1.0))
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
X = np.arange(-1,7)
#what is X2 numbers probably aren't right for this
X2 = np.array([0,0,0,.2,.4,.8,1,1.2,])
plt.plot(X, 100/(1+np.exp(.5*(3-X+np.random.uniform(size=8)))), color='blue')
plt.plot(X, 100/(1+np.exp(.5*(3-X+4*X2+np.random.uniform(size=8)))), color='black')
plt.plot(X, 100/(1+np.exp(.5*(3-X+4*X2+np.random.uniform(size=8)))), color='green')
plt.title("Hypothetical conditional effects plot, biased against high-risk black applicants")
plt.ylabel('Loan Origination Status')
plt.xlabel('Applicant Income Bracket')
plt.xlim(0,max(X) + 1)
plt.ylim(0,100)
ax.legend(["White", "Black or AFAM", "Asian"], bbox_to_anchor=(1.4, 1.0))
plt.show()

### Using this data to automate loan approval

Now, using the 2015 New York State loan data, we'll train a logistic regression classifier to determine whether this apparent bias against African Americans (when we control solely for income) would still appear in a classifier that ideally would control for additional factors. In training this classifier, we'll exclude characteristics that are explicitly racially focused (race of applicant name, fraction of neighborhood that is a minority population, etc).  While there are some characteristics that correlate with race such as census tract, we will include those because they're not explcitly racial and are on mortgage applications – after all, lenders need to know where the houses they are providing loans for are located. 

In [ ]:
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as Logit
from sklearn.linear_model import LinearRegression as OLS
import sys
import pickle

In [ ]:
np.array(sorted(df_test.columns))

Model: 
action_taken_name ~ applicant_income_000s + hud_median_family_income +  loan_amount_000s + number_of_1_to_4_family_units + number_of_owner_occupied_units + population + I\[agency_abbr\] + I\[census_tract_number\] + I\[income_bracket\] + I\[lien_status_name\] + I\[loan_purpose_name\] + I\[loan_type_name\] + I\[owner_occupancy_name\] + I\[purchaser_type_name\]

Excluded variables: 

* agency_name: redundant with agency_abbr
* (co\_)applicant\_\[ethnicity | race\]\_name\_\[1|2|3|4|5\]: we're excluding explicitly racial characteristics from the regression
* application_date_indicator: unlikely the time at which one applies should influence one's outcome
* as_of_year: 2015 for all data points
* census_tract_number: people's home/address is often highly correlated with their race so we will exclude this from the analysis.
* county_name: census_tract is more specific
* denial\_reason\_\[1|2|3\]: if this is not nan then the applicant was rejected and so it predicts the outcome almost always
* hoepa_status_name: in all but 56 out of ~372k records this is "Not a HOEPA loan". Excluded b/c uninformative
* minority_population: too much of a racial proxy
* msamd_name: captured by the census tracts
* preapproval_name: almost always nan
* property_type_name: always "One-to-four family dwelling"
* rate_spread: almost always nan and would be important but not populated until 2018 (unreleased) HMDA data set.
* respondent_id: irrelevant. This is the lender's unique id
* sequence_number: irrelevant. This is the loan id number
* state_abbr: always "NY"
* state_name: always "New York"
* tract_to_msamd_income: if we include hud_median_family_income, tract_to_msamd_income is redundant because it says how many times greater a tract's income is relative to the mediann of the metro area, while hud captures the median income of the area. So they're highly correlated

A variable that is denoted  I\[$\cdot$\] is a discrete variable and so each of the possible values the categories could take on will be represented with an indicator variable.  All of the continuous variables are Z-Scored so that the result and coefficients of the logistic regression are interpretable. 

In [ ]:
# One hot encoding converts categorical variables into a form that is better for ML. 
# Specific value types are unique for each category.
def oneHotFromCategory(rowValue, uniqueCategoryValues):
    return np.array(list(map(lambda x: int(x == rowValue), uniqueCategoryValues)))

In [ ]:
continuousVars = ["applicant_income_000s", "hud_median_family_income", "loan_amount_000s",
               "number_of_1_to_4_family_units", "number_of_owner_occupied_units", "population"]

discreteVars = ["agency_abbr", "income_bracket", "lien_status_name", "loan_purpose_name", 
                "loan_type_name", "owner_occupancy_name", "purchaser_type_name"]

In [ ]:
# logisticData = df_test[["action_taken_name"] + continuousVars + discreteVars].copy()
# n0 = len(logisticData)
# logisticData = logisticData.dropna()
# for var in set(discreteVars) - set(['census_tract_number', 'income_bracket']):
#     logisticData = logisticData[logisticData[var] != 'nan']
# n1 = len(logisticData)
# print("Start: %d\tComplete: %d\tDropped: %d (%.2f%%)" % (n0, n1, n0 - n1, 100 * (n0 - n1) / float(n0)))

# # # Z-Score Continuous Variables
# for var in continuousVars:
#     logisticData[[var]] = (logisticData[[var]] - logisticData[[var]].mean()) /  logisticData[[var]].std()
    
# # Add Dummy Variables for Discrete Variables
# addDummiesFor = sorted(list(set(discreteVars) - set(["census_tract_number"])))
# for var in addDummiesFor:
#     sys.stdout.write("Starting: "); sys.stdout.write(var); sys.stdout.write("\t")
#     u = sorted(logisticData[var].unique())
#     r = logisticData.apply(lambda x: oneHotFromCategory(x[var], u), axis = 1)
#     out = np.array([i for i in r.values])
#     logisticData = logisticData.join(pd.DataFrame(out, columns = u, index = logisticData.index))
#     sys.stdout.write("Finished: "); sys.stdout.write(var); sys.stdout.write("\n")

# with open('logisticData_with_Dummies.pickle', 'wb') as f:
#     pickle.dump(logisticData, f, pickle.HIGHEST_PROTOCOL)

with open('logisticData_with_Dummies.pickle', 'rb') as f:
    logisticData = pickle.load(f)

In [ ]:
# excluding census tract data so don't do this
# logisticData = logisticData.set_index("census_tract_number")
# logisticData["mean_tract_outcome"] = logisticData.groupby("census_tract_number").action_taken_name.mean()
# logisticData = logisticData.reset_index()
# logisticData["transformed_outcome"] = logisticData.action_taken_name - logisticData.mean_tract_outcome

In [ ]:
logisticData = logisticData.rename({
    1.0 : "IncomeLevel_1",
    2.0 : "IncomeLevel_2",
    3.0 : "IncomeLevel_3",
    4.0 : "IncomeLevel_4",
    5.0 : "IncomeLevel_5",
    6.0 : "IncomeLevel_6",
    7.0 : "IncomeLevel_7",
    "Not secured by a lien" : "LienUnsecured",
    "Secured by a first lien" : "FirstLienSecured",
    "Secured by a subordinate lien" : "SubordinateLienSecured",
    "Home improvement" : "HomeImprovement",
    "Home purchase" : "HomePurchase",
    "FHA-insured" : "FHA_insured",
    "FSA/RHS-guaranteed" : "FSA_RHS_guaranteed",
    'VA-guaranteed' : 'VA_guaranteed',
    'Not owner-occupied as a principal dwelling' : "NotOwnerOccupied",
    'Owner-occupied as a principal dwelling' : "OwnerOccupied",
    'Affiliate institution' : "AffiliateInstitution",
    'Commercial bank, savings bank or savings association' : "CommercialBank",
    'Fannie Mae (FNMA)' : "FannieMae",
    'Farmer Mac (FAMC)' : "FarmerMac",
    'Freddie Mac (FHLMC)' : "FreddieMac",
    'Ginnie Mae (GNMA)' : "GinnieMae",
    'Life insurance company, credit union, mortgage bank, or finance company' : "LifeInsurance",
    'Loan was not originated or was not sold in calendar year covered by register' : "WrongYear",
    'Other type of purchaser' : "OtherPurchaser",
    'Private securitization' : "PrivateSecuritization"
}, axis = 1)

In [ ]:
np.random.seed(4321)
trainData, testData = train_test_split(logisticData, test_size=0.2)

X_train = trainData.drop(discreteVars + ["action_taken_name"] , axis = 1)
y_train_binary = trainData.action_taken_name # does not account for census_tract effects
# y_train_cts = trainData.transformed_outcome

X_test = testData.drop(discreteVars + ["action_taken_name"] , axis = 1)
y_test_binary = testData.action_taken_name # does not account for census_tract effects
# y_test_cts = testData.transformed_outcome

In [ ]:
binary_model = Logit(solver = "lbfgs", max_iter = 1000)
binary_model.fit(X_train.values, y_train_binary.values)

In [ ]:
logit_predictions = binary_model.predict(X_test.values)

sum(list(map(lambda x: int(x), (logit_predictions == y_test_binary)))) / len(X_test)

In [ ]:
def isNaN(x):
    return x != x

In [ ]:
originalWithProbs = df_test\
.join(pd.DataFrame(binary_model.predict_proba(X_test)[:,1], 
                   columns = ["probOfAcceptance"], index = X_test.index))
originalWithProbs = originalWithProbs[~isNaN(originalWithProbs.probOfAcceptance)]

To see if our classifier is biased against certain races, we will group the predictions for each racial group into 10 groups [0,0.1], ... [0.9,1.0] and then see if for each group, the true probability of receiving a loan varied by race. 

In [ ]:
raceAndProbData = originalWithProbs[originalWithProbs.applicant_race_name_1.isin([Race.Asian, Race.Black, Race.White])]\
                  [["applicant_race_name_1", "probOfAcceptance", "action_taken_name"]]\
                  .sort_values(["probOfAcceptance"], ascending = False)

raceAndProbData["probBucket"] = raceAndProbData.probOfAcceptance.apply(lambda x: round((x + 0.0499999999) * 10))
means = raceAndProbData\
.groupby(["applicant_race_name_1", "probBucket"])\
.action_taken_name\
.mean()\
.reset_index(name = "meanAcceptance")\
.set_index(["applicant_race_name_1", "probBucket"])

raceAndProbData = raceAndProbData.set_index(["applicant_race_name_1", "probBucket"])
raceAndProbData["meanAcceptance"] = means.meanAcceptance
raceAndProbData = raceAndProbData.reset_index()

raceAndProbData["predictedOutcome"] = raceAndProbData.probOfAcceptance.apply(lambda x: int(x >= 0.5))

del(means)

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

asianData = raceAndProbData[raceAndProbData.applicant_race_name_1 == Race.Asian][["probBucket", "meanAcceptance"]].drop_duplicates().reset_index()
blackData = raceAndProbData[raceAndProbData.applicant_race_name_1 == Race.Black][["probBucket", "meanAcceptance"]].drop_duplicates().reset_index()
whiteData = raceAndProbData[raceAndProbData.applicant_race_name_1 == Race.White][["probBucket", "meanAcceptance"]].drop_duplicates().reset_index()

plt.scatter(asianData.probBucket, asianData.meanAcceptance, label = "Asian", color = 'g')
plt.scatter(blackData.probBucket, blackData.meanAcceptance, label = "Black", color = 'k')
plt.scatter(whiteData.probBucket, whiteData.meanAcceptance, label = "White", color = 'b')

plt.xlabel("Predicted Probability of Acceptance")
plt.ylabel("True Probability of Acceptance")
plt.xticks(range(1,11))
ax.legend(['Asian', 'Black or African American', 'White'], bbox_to_anchor=(1.1, 1.0))
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

plt.scatter(asianData.probBucket, asianData.meanAcceptance - whiteData.meanAcceptance, label = "Asian", color = 'g')
plt.scatter(blackData.probBucket, blackData.meanAcceptance - whiteData.meanAcceptance, label = "Black", color = 'k')

plt.xlabel("Predicted Probability of Acceptance")
plt.ylabel("Difference in Acceptance Rel. TO Whites")
ax.legend(['Asian', 'Black or African American'], bbox_to_anchor=(1.1, 1.0))
plt.title("True Acceptance Probability Relative to Whites")
ax.axhline(y = 0, color = 'r')
plt.xticks(range(1,11))
plt.show()

### Confusion Matrix

In [ ]:
def getConfusionMatrix(racesToInclude = list(raceAndProbData.applicant_race_name_1.unique())):
    actual_pos = (raceAndProbData.action_taken_name == 1.0); actual_neg = (raceAndProbData.action_taken_name == 0.0);
    pred_pos = (raceAndProbData.predictedOutcome == 1.0); pred_neg = (raceAndProbData.predictedOutcome == 0.0);
    
    racialFilter = raceAndProbData.applicant_race_name_1.isin(racesToInclude)

    true_positives = len(raceAndProbData[actual_pos & pred_pos & racialFilter])
    true_negatives = len(raceAndProbData[actual_neg & pred_neg & racialFilter])
    false_positives = len(raceAndProbData[actual_neg & pred_pos & racialFilter])
    false_negatives = len(raceAndProbData[actual_pos & pred_neg & racialFilter])

    TPR = true_positives / (true_positives + false_negatives)
    FPR = false_positives / (true_negatives + false_positives) # 1 - TNR
    TNR = true_negatives / (true_negatives + false_positives)
    FNR = false_negatives / (true_positives + false_negatives) # 1 - TPR

    print("""
            Act.Pos | Act.Neg 
          #-------------------#
          |         |         |
    Pred. |  (TPR)  |  (FPR)  |
    Pos   |  %0.3f  |  %0.3f  | 
          |         |         |
    ------|---------#---------|
          |         |         |
    Pred. |  (FNR)  |  (TNR)  |
    Neg   |  %0.3f  |  %0.3f  |
          |         |         |
          #-------------------#
    """ % (TPR, FPR, FNR, TNR))

In [ ]:
getConfusionMatrix()

In [ ]:
getConfusionMatrix([Race.Asian])

In [ ]:
getConfusionMatrix([Race.Black])

In [ ]:
getConfusionMatrix([Race.White])

### Conclusion
Our classifier treats Whites and Asians virtually identically from a confusion matrix perspective. However, our classifier is less likely to "mistakenly" approve a black applicant for a loan (the False Positive Rate for Blacks is 18%, compared to 33% for whites ) and more likely to "mistakenly" reject a black applicant for a loan (the False Negative Rate for Blacks is 11%, compared to 8% for Whites). These discrepancies are most prevalent in the ~50% porbabiltiy of loan origination/purchased by an institution range.